In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv('insurance.csv')

In [3]:
df.head(1)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.9,0,yes,southwest,16884.924


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
# Replace 'your_database.db' with the name of your database file
conn = sqlite3.connect('insurance.db')
conn


In [6]:
# Replace 'table_name' with the name you want for the table
# df.to_sql('insurance', conn, if_exists='replace', index=False)


In [7]:
query = "SELECT * FROM insurance LIMIT 5;"
result = pd.read_sql(query, conn)
print(result)

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [8]:
def sql_query(q):
    return pd.read_sql_query(q, conn)

In [9]:
sql_query("select * from sqlite_master")

,type,name,tbl_name,rootpage,sql
0,table,insurance,insurance,2,"CREATE TABLE ""insurance"" (\n""age"" INTEGER,\n ..."


### ***1. Select all columns for all patients.***

In [10]:
sql_query('select * from insurance')

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## ***2. Display the average claim amount for patients in each region.***

In [11]:
sql_query('''select avg(charges) as Avg_claim, region from insurance
            group by region
'''
)

,Avg_claim,region
0,13406.384516,northeast
1,12417.575374,northwest
2,14735.411438,southeast
3,12346.937377,southwest


### ***3. Select the maximum and minimum BMI values in the table.***

In [12]:
sql_query("Select max(bmi) as max_bmi ,min(bmi) as min_bmi from insurance"
)

,max_bmi,min_bmi
0,53.13,15.96


### ***4. Select the age, and BMI for patients with a BMI between 40 and 50.***

In [13]:
sql_query('''select age,bmi from insurance
            where bmi between 40 and 50
''')

,age,bmi
0,27,42.13
1,56,40.30
2,48,41.23
3,58,49.06
4,63,41.47
...,...,...
83,25,42.13
84,52,41.80
85,34,42.90
86,42,40.37


### ***5. Select the number of smokers in each region.***

In [14]:
sql_query('''select count(smoker) as smoker, region from insurance
            where smoker == 'yes'
            group by region

''')

,smoker,region
0,67,northeast
1,58,northwest
2,91,southeast
3,58,southwest


### ***6. Retrieve all patients who have a BMI greater than the average BMI of patients who are smokers.***

In [15]:
sql_query('select avg(bmi) from insurance')

,avg(bmi)
0,30.663397


In [16]:
sql_query('''select * from insurance
            where bmi > (select avg(bmi) from insurance) and smoker == 'yes'
''')

,age,sex,bmi,children,smoker,region,charges
0,27,male,42.130,0,yes,southeast,39611.75770
1,30,male,35.300,0,yes,southwest,36837.46700
2,34,female,31.920,1,yes,northeast,37701.87680
3,31,male,36.300,2,yes,southwest,38711.00000
4,22,male,35.600,0,yes,southwest,35585.57600
...,...,...,...,...,...,...,...
130,20,male,39.400,2,yes,southwest,38344.56600
131,19,male,34.900,0,yes,southwest,34828.65400
132,62,male,30.875,3,yes,northwest,46718.16325
133,19,female,34.700,2,yes,southwest,36397.57600


### ***8. Select the average charges amount for patients in each age group.***

In [20]:
sql_query("""
    select 
	case when age < 18 then "Under 18"
    when age between 18 and 30 then "18-30"
    when age between 31 and 50 then "31-50"
    else "Over 50"
    end as age_group,
    round(avg(charges),2) as avg_charges
    from insurance
    group by age_group

""")

,age_group,avg_charges
0,18-30,9397.55
1,31-50,13280.77
2,Over 50,18084.99
